In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
## Loading the Imdb dataset's word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [5]:
model = load_model('imdb_rnn.keras')
model.summary()

I0000 00:00:1741346771.665140   43386 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,939,077 (15.03 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,052 (10.02 MB)

In [6]:
model.get_weights()

[array([[-0.19458643, -0.04357468, -0.03960358, ..., -0.03857193,
         -0.02123128,  0.01810508],
        [-0.01882016,  0.0002039 ,  0.01377379, ..., -0.02544832,
          0.02362547, -0.00030763],
        [-0.16626264,  0.03738732, -0.06280185, ..., -0.02379228,
         -0.08232426,  0.14336137],
        ...,
        [-0.01052773, -0.0422865 ,  0.06708509, ...,  0.08842884,
         -0.08965087, -0.09595361],
        [ 0.00196209,  0.04819118,  0.04197874, ..., -0.01069223,
         -0.0413865 ,  0.02299897],
        [ 0.11405707, -0.11373773,  0.08144619, ...,  0.13752347,
          0.05199527, -0.1127319 ]], dtype=float32),
 array([[-0.06007943,  0.07118436,  0.06591284, ...,  0.16182384,
         -0.09123229,  0.03568731],
        [-0.13054374, -0.02027185,  0.13606659, ..., -0.2474176 ,
          0.07821142, -0.10040452],
        [-0.05808602, -0.1417938 ,  0.1502438 , ...,  0.08013663,
         -0.08014653,  0.09522058],
        ...,
        [ 0.08247584,  0.0679764 , -0.0

In [7]:
## function to decode the reviews

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

In [8]:
## function to preprocess the user text

def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen = 500)
    return padded_review

In [9]:
## Prediction function
def predict_sentiment(review):
    pp_inp = preprocess_text(review)
    prediction = model.predict(pp_inp)
    
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]

In [10]:
example_review = "This movie was fantastic and the acting was great. Th plot was thrilling and kept the viewers on their feet"

sentiment, score = predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
Review: This movie was fantastic and the acting was great. Th plot was thrilling and kept the viewers on their feet
Sentiment: Positive
Prediction Score: 0.7662276029586792


I0000 00:00:1741347412.291732   43989 service.cc:148] XLA service 0x7f8eec003c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741347412.291787   43989 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-03-07 12:36:52.302650: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741347412.322495   43989 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1741347412.442941   43989 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
